In [117]:
!pip install mibian

In [43]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

  # Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
%matplotlib inline
warnings.filterwarnings('ignore')
####example
#import tools
#tools.compile_df(stock_final, 'BIG')

In [92]:
###this is the small sample pickle, use this for testing

df = pd.read_pickle('list_tries.pickle')
# df=df.drop(df.columns[0], axis=1)
display(df.columns)
#pip install yfinance
start = '2018-10-01'
end = '2021-01-13'
Symbols=['TSLA', 'AAPL', 'AMD', 'JETS', 'AMC', 'SAVE', 'SHOP', 'GE', 'TSM', 'ACB', 'TWTR', 'PFE', 'F', 'WFC', 'ATVI', 'DIS', 'CCL', 'SLV', 'SPCE', 'NAKD', 'NOK', 'INO', 'GOLD', 'EBAY', 'BIG',  ]
#data = yf.download(symbol, start_date, end_date)

Index(['optionkey', 'Symbol', 'ExpirationDate', 'AskPrice', 'AskSize',
       'BidPrice', 'BidSize', 'LastPrice', 'PutCall', 'StrikePrice', 'Volume',
       'openinterest', 'UnderlyingPrice', 'DataDate'],
      dtype='object')

In [53]:
#this is the larger dataset

# df = pd.read_csv('try_this.csv')
# #Drop the unnamed column
# #df=df.drop(df.columns[0], axis=1)
# display(df.columns)
# df
# #pip install yfinance
# start = '2016-01-01'
# end = '2020-01-31'
# Symbols=['TSLA', 'AAPL', 'AMD', 'JETS', 'SHOP', 'TWTR', 'ATVI', 'DIS', 'CCL', 'GOLD'  ]
# #data = yf.download(symbol, start_date, end_date)

Index(['optionkey', 'Symbol', 'ExpirationDate', 'AskPrice', 'AskSize',
       'BidPrice', 'BidSize', 'LastPrice', 'PutCall', 'StrikePrice', 'Volume',
       'openinterest', 'UnderlyingPrice', 'DataDate', 'time delta'],
      dtype='object')

In [93]:
#the pipeline to get the features dataframe
#ov= option volume dataframe
#uv=underlying volume dataframe
#run this cell after running the functions below

mf={}
def pipeline(ov, uv):
    for each_symbol in stock_final['Name'].unique():
        ov = feature_engineering(ov)
        opt = get_option_volume(ov, each_symbol)
        undv = underlying_volume(uv, each_symbol)
        combined = merge(opt, undv)
        mf[each_symbol]=combined
        #save each of the features dataframes to csv
        combined.to_csv(f'{each_symbol}_initial.csv')
    return mf

features = pipeline(df, stock_final)
features.keys()
tsla_features = features['TSLA']

In [94]:
tsla_features

,Call Volume,Put Volume,Total Volume,Percent of cPut,Put signals,Percent of calls,Call signals,Option rVol,option rvol delta,Close,...,bollinger_signal,rvol,rvol SD,rvol signal,daily returns,volume delta,bb std delta,rvol delta,Option rVol SD,Option rVol signal
2016-02-26,61082.0,55707.0,116789.0,0.476988,False,0.523012,False,1.265347,0.009447,38.068001,...,0.0,0.918543,1.476481,0.0,0.015526,0.000000,0.000000,0.000000,NaN,0.0
2016-02-29,38760.0,33531.0,72291.0,0.463834,False,0.536166,False,0.765316,-0.395173,38.386002,...,0.0,0.672970,1.475945,0.0,0.008353,-0.258215,-0.612534,-0.267351,NaN,0.0
2016-03-01,46761.0,66906.0,113667.0,0.588614,False,0.411386,False,1.126356,0.471753,37.270000,...,1.0,0.993514,1.464062,0.0,-0.029073,0.491932,0.259721,0.476312,0.945510,0.0
2016-03-02,36792.0,43244.0,80036.0,0.540307,False,0.459693,False,0.902682,-0.198582,37.667999,...,0.0,0.724600,1.435430,0.0,0.010679,-0.275588,-0.016266,-0.270669,0.883082,0.0
2016-03-03,51158.0,52002.0,103160.0,0.504091,False,0.495909,False,1.042501,0.154893,39.147999,...,-1.0,0.736651,1.375740,0.0,0.039291,-0.006869,0.749635,0.016632,0.894204,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-24,297731.0,265006.0,562737.0,0.470923,False,0.529077,False,0.770595,-0.027754,112.963997,...,1.0,0.791383,1.555173,0.0,-0.012898,-0.269574,-0.727745,-0.282258,1.484651,0.0
2020-01-27,179882.0,130124.0,310006.0,0.419747,False,0.580253,False,0.633703,-0.177645,111.603996,...,0.0,0.744179,1.542626,0.0,-0.012039,-0.051938,0.896518,-0.059648,0.976759,0.0
2020-01-28,137593.0,110051.0,247644.0,0.444392,False,0.555608,False,0.663103,0.046394,113.379997,...,0.0,0.641439,1.531559,0.0,0.015913,-0.133714,-0.345125,-0.138058,0.764001,0.0
2020-01-29,281369.0,246957.0,528326.0,0.467433,False,0.532567,False,1.459496,1.201010,116.197998,...,-1.0,0.955069,1.518071,0.0,0.024854,0.510073,1.494002,0.488948,0.860175,0.0


In [95]:
tsla_signals= get_signals(tsla_features)
tsla_signals

,volume delta,bb std delta,rvol delta,option rvol delta,daily returns
2016-02-26,NaN,NaN,NaN,NaN,0.015526
2016-02-29,-0.258215,-0.612534,-0.267351,-0.395173,0.008353
2016-03-01,0.491932,0.259721,0.476312,0.471753,-0.029073
2016-03-02,-0.275588,-0.016266,-0.270669,-0.198582,0.010679
2016-03-03,-0.006869,0.749635,0.016632,0.154893,0.039291
...,...,...,...,...,...
2020-01-24,-0.269574,-0.727745,-0.282258,-0.027754,-0.012898
2020-01-27,-0.051938,0.896518,-0.059648,-0.177645,-0.012039
2020-01-28,-0.133714,-0.345125,-0.138058,0.046394,0.015913
2020-01-29,0.510073,1.494002,0.488948,1.201010,0.024854


In [96]:
tsla_ltsm = lstm(tsla_signals, 7)
tsla_ltsm

ValueError: in user code:

    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\austi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

In [54]:
####Get the underlying stock prices and volume by date

def get_underlying(Symbols):
# create empty dataframe
    stock_final = pd.DataFrame()
# iterate over each symbol
    for i in Symbols:  


        try:
            # download the stock price 
            stock = []
            stock = yf.download(i,start=start, end=end, progress=False)

            # append the individual stock prices 
            if len(stock) == 0:
                None
            else:
                stock['Name']=i
                stock_final = stock_final.append(stock,sort=False)
                stock_final= stock_final.drop(['Open', 'High', 'Low', 'Adj Close'], axis=1)
        except Exception:
            None
    return stock_final

stock_final = get_underlying(Symbols)

In [83]:
df['Symbol'].value_counts()

TSLA    2267122
AAPL    1506921
DIS      946721
SHOP     840306
ATVI     827300
TWTR     819018
AMD      684704
CCL      600114
GOLD     414462
JETS     227878
Name: Symbol, dtype: int64

In [8]:
### To slice the dataframe for a particular symbol or option type

def select_df(df, symbol, option):
    df = df.loc[df['PutCall'] == option]
    df = df.loc[df['Symbol'] == symbol]
    return df


In [7]:
###The first process, for the first features needed for the program

def feature_engineering(data):
    #for i in data:
    #data['ExpirationDate'] = data['ExpirationDate'].apply(pd.Timestamp)
    #data['DataDate'] = data['DataDate'].apply(pd.Timestamp)
    #data = data.loc[data['PutCall'] == 'put']

    data['ExpirationDate'] =  pd.to_datetime(data['ExpirationDate'], infer_datetime_format=True)
    data['DataDate'] =  pd.to_datetime(data['DataDate'], infer_datetime_format=True)
    data['time delta'] = data['ExpirationDate'] - data['DataDate']
    data['StrikePrice'] = pd.to_numeric(data['StrikePrice'])
    data['UnderlyingPrice'] = pd.to_numeric(data['UnderlyingPrice'])
    data['% out'] = data['StrikePrice'] / data['UnderlyingPrice'] - 1
    data['exp?'] =data['ExpirationDate'] ==data['DataDate']
    #data['ExpirationDate']= data['ExpirationDate'].to_string
    #data['DataDate'] = data['DataDate'].to_string
    data['DateKey'] = data["DataDate"].astype(str) + data["ExpirationDate"].astype(str) 

    #data['xp'] = data.loc[data['exp?'] == True, 'AskPrice']
    #data['xp']=data.OptionKey.map(xp.set_index('OptionKey')['xp'])

    return data



In [85]:
###To map the expiration value by optionkey, so it will appear on all 
###df.loc[df'AskPrice'] > df['xp']
def map_exp(df):
    xp = df
    xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
    xp = xp.loc[xp['exp?']== True]
    xp= xp[['optionkey', 'xp']]
    df['xp']=df.optionkey.map(xp.set_index('optionkey')['xp'])
    return df

In [11]:
###Pass the data frame with features in to this

def get_option_volume(option_df, symbol):
    #Calculate volume by date
    mf=option_df[option_df['Symbol']==symbol]
    mf['Volume'] = mf['Volume'].astype(int)
    mf['Total Volume'] = mf.groupby(['DataDate']).Volume.transform('sum')
    mf['Call Volume'] = mf.loc[mf['PutCall'] == 'call', 'Volume']
    mf['Put Volume'] = mf.loc[mf['PutCall'] == 'put', 'Volume']
    volume = mf.groupby('DataDate').agg({'Call Volume': 'sum', 'Put Volume' : 'sum'})
    volume['Total Volume'] = volume.loc[:,['Call Volume','Put Volume']].sum(axis=1)

    #Create put signals
    volume['Percent of cPut'] = volume['Put Volume']/volume['Total Volume']
    volume['Put signals'] = volume['Percent of cPut'] > .6

    #Create Call volume
    volume['Percent of calls'] = volume['Call Volume']/volume['Total Volume']
    volume['Call signals'] = volume['Percent of calls'] > .6
    volume_ratio = 3 #20
    volume['Option rVol'] = volume['Total Volume']/ volume['Total Volume'].rolling(window = volume_ratio).mean()
    

    volume['option rvol delta'] = volume['Option rVol'].dropna().pct_change()

    return volume


In [87]:
### Pass the stock underlying dataframe in this


def get_underlying_volume(uf, symbol):

    bollinger_window= 3 #20
    
    volume_df=uf[uf['Name']==symbol]
    #volume_df = volume_df.merge(volume, left_index=True, right_index = True)

     # more steps feature engineering
    volume_df.loc[:, 'bollinger_mid_band'] = volume_df['Close'].rolling(window=bollinger_window).mean()
    volume_df.loc[:, 'bollinger_std'] = volume_df['Close'].rolling(window=bollinger_window).std()
    
     #Calculate upper and lowers bands of bollinger band
     
    volume_df['bollinger_upper_band']  = volume_df['bollinger_mid_band'] + (volume_df['bollinger_std'] * 1)
    volume_df['bollinger_lower_band']  = volume_df['bollinger_mid_band'] - (volume_df['bollinger_std'] * 1)

     #Calculate bollinger band trading signal
    volume_df['bollinger_long'] = np.where(volume_df['Close'] < volume_df['bollinger_lower_band'], 1.0, 0.0)
    volume_df['bollinger_short'] = np.where(volume_df['Close'] > volume_df['bollinger_upper_band'], -1.0, 0.0)
    volume_df['bollinger_signal'] = volume_df['bollinger_long'] + volume_df['bollinger_short']
     #construct relative volume ratio and append to volume
    volume_ratio = 20

     #calculate rvol and 1 deviation of a 20 day rvol
    volume_df['rvol'] = volume_df['Volume']/ volume_df['Volume'].rolling(window = volume_ratio).mean()
    volume_df['rvol SD'] = volume_df['rvol'].rolling(window = volume_ratio).std()/2+ volume_df['rvol'].rolling(window = volume_ratio).mean()
#     #create rvol signal where rvol > 1 SD rvol
    volume_df['rvol signal'] = np.where(volume_df['rvol'] > volume_df['rvol SD'], 1.0, 0.0)

     #create options rVol signal
    volume_ratio = 20

    #volume_df['Option rVol'] = volume_df['Total Volume']/ volume_df['Total Volume'].rolling(window = volume_ratio).mean()
    #volume_df['Option rVol SD'] = volume_df['rvol'].rolling(window = volume_ratio).std()/2+ volume_df['rvol'].rolling(window = volume_ratio).mean()
     #create rvol signal where rvol > 1 SD rvol
    #volume_df['Option rVol signal'] = np.where(volume_df['rvol'] > volume_df['rvol SD'], 1.0, 0.0)

     #create daily returns column and Actual Value column
    volume_df['daily returns'] = volume_df['Close'].dropna().pct_change()

     #drop nans due to BB/rvol calculations
    volume_df = volume_df.dropna()
   # return volume_df
    
    trading_signals = volume_df
    trading_signals['volume delta'] = volume_df['Volume'].dropna().pct_change()
    trading_signals['bb std delta'] = volume_df['bollinger_std'].dropna().pct_change()
    trading_signals['rvol delta'] = volume_df['rvol'].dropna().pct_change()
#     #add daily returns as target
    trading_signals['daily returns'] = volume_df['daily returns'].dropna()

    trading_signals= trading_signals.fillna(value = 0)
    trading_signals= trading_signals.replace([np.inf, -np.inf], 0.0)
    
    trading_signals.head()
    return trading_signals

In [88]:
###Merge the option and underlying volume dfs, and create extra signals

def merge(uv,ov):
    combined = uv.merge(ov, left_index = True, right_index = True, how = 'inner')
    volume_ratio = 3 #20
    combined['Option rVol SD'] = combined['rvol'].rolling(window = volume_ratio).std()/2+ combined['rvol'].rolling(window = volume_ratio).mean()

    combined['rvol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)

    combined['Option rVol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)
    return combined

#trading_signals = merge(underlying_volume, option_volume)


In [86]:
###Creating spreads, not necessary for this strategey


def create_spreads(data):
    #Sort Values and shift to set up collateral requirements
    data['DateKey']=data['DataDate'].astype('str')+data['ExpirationDate'].astype('str') 
    data.groupby('DateKey') 
    data.sort_values('DataDate') 
    data['Prev_Strike']=data['StrikePrice'].shift(1)
    data['Prev_AskPrice']=data['AskPrice'].shift(1)
    data['Prev_DateKey']=data['DateKey'].shift(1)

    # Create nan values where the shifted value uses a different day
    for i in data.index:
        if data.loc[i, 'Prev_DateKey']!=data.loc[i, 'DateKey']:
            data.loc[i, 'Prev_Strike']=np.nan
            data.loc[i, 'Prev_AskPrice']=np.nan
            
    return data

In [90]:
tsla_features = check['TSLA']
tsla_signals = signals(tsla_features)
tsla_signals

,volume delta,bb std delta,rvol delta,option rvol delta,daily returns
2016-02-26,NaN,NaN,NaN,NaN,0.015526
2016-02-29,-0.258215,-0.612534,-0.267351,-0.395173,0.008353
2016-03-01,0.491932,0.259721,0.476312,0.471753,-0.029073
2016-03-02,-0.275588,-0.016266,-0.270669,-0.198582,0.010679
2016-03-03,-0.006869,0.749635,0.016632,0.154893,0.039291
...,...,...,...,...,...
2020-01-24,-0.269574,-0.727745,-0.282258,-0.027754,-0.012898
2020-01-27,-0.051938,0.896518,-0.059648,-0.177645,-0.012039
2020-01-28,-0.133714,-0.345125,-0.138058,0.046394,0.015913
2020-01-29,0.510073,1.494002,0.488948,1.201010,0.024854


In [74]:
tsla_lstm = lstm(tsla_signals, 7)
tsla_lstm

,Actual,Predicted,Positive Actual signal,Negative Actual signal,Positive Predicted signal,Negative Predicted signal,Actual Signal,LSTM Predicted Signal
2020-01-21,0.504339,NaN,1,0,0,0,1,0


In [45]:
features = np.asarray(tsla_features)

In [40]:
tsla_features = tsla_features.drop(columns = ['Name'])

In [79]:
###Create the signals that will be passed to LSTM
###This result is supposed to work in LSTM, however it isnt right now


def get_signals(features):
    signals_df = pd.DataFrame()
    signals_df['volume delta'] = features['Volume'].dropna().pct_change()
    signals_df['bb std delta'] = features['bollinger_std'].dropna().pct_change()
    signals_df['rvol delta'] = features['rvol'].dropna().pct_change()
    signals_df['option rvol delta'] = features['Option rVol'].dropna().pct_change()
    signals_df['daily returns'] = features['daily returns'].dropna()

    return signals_df


In [91]:
###LSTM functino

def window_data(window):
    """
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    
    """
       
    X = []
    y = []
    for i in range(len(trading_signals_df) - window):
        features = trading_signals_df.iloc[i : (i + window), :]
        target = trading_signals_df.iloc[(i + window), -1]
        X.append(features)
        y.append(target)
    
    return np.array(X), np.array(y).reshape(-1, 1)

def lstm(trading_signals_df, window_size):
        # Define the window size

    # Create the features (X) and target (y) data using the window_data() function.
    X, y = window_data(window_size)

    # Manually splitting the data
    split = int(0.7 * len(X))

    X_train = X[: split]
    X_test = X[split:]

    y_train = y[: split]
    y_test = y[split:]

    scalers = {}
    for i in range(X_train.shape[1]):
        scalers[i] = MinMaxScaler()
        X_train[:, i, :] = scalers[i].fit_transform(X_train[:, i, :]) 

    for i in range(X_test.shape[1]):
        X_test[:, i, :] = scalers[i].transform(X_test[:, i, :])

    scaler = MinMaxScaler()
    scaler.fit(y)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

    # Define the Transformer model.
    model = Sequential()

    # Initial model setup
    number_units = 7
    dropout_fraction = 0.3
    inputs = len(trading_signals_df.columns)

    # Layer 1
    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X.shape[1], inputs))
        )
    model.add(Dropout(dropout_fraction))

    # Layer 2
    model.add(LSTM(units=number_units, return_sequences=True))
    model.add(Dropout(dropout_fraction))

    # Layer 3
    model.add(LSTM(units=number_units))
    model.add(Dropout(dropout_fraction))

    # Output layer
    model.add(Dense(1, activation = 'linear'))

    # Compile the LSTM model
    model.compile(optimizer="adam", loss="mean_squared_error")

     # Train the model
    model.fit(X_train, y_train, epochs=100, shuffle=False, verbose=0)

     # Make predictions using the testing data X_test
    predicted = model.predict(X_test)

    #Recover the original, nonscaled prices
    predicted_prices = scaler.inverse_transform(predicted)
    real_prices = scaler.inverse_transform(y_test.reshape(-1,1)) 

    # Create a DataFrame of Real and Predicted values, shifted by forward window
    shift =7
    stocks = pd.DataFrame({
        "Actual": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = trading_signals_df.index[-len(real_prices)-shift:-shift ] )


    # Show the DataFrame's head
    stocks

    #convert stocks df into positive and negative signals
    stocks['Positive Actual signal'] = np.where(stocks['Actual'] > 0, 1, 0)
    stocks['Negative Actual signal'] = np.where(stocks['Actual'] < 0, -1, 0)

    stocks['Positive Predicted signal'] = np.where(stocks['Predicted'] > 0, 1, 0)
    stocks['Negative Predicted signal'] = np.where(stocks['Predicted'] < 0, -1, 0)

    #merge to create one column per signal, shifted back to reflect forward projection window
    shift = 7
    stocks['Actual Signal'] = stocks['Positive Actual signal'] + stocks['Negative Actual signal']
    stocks['LSTM Predicted Signal'] = stocks['Positive Predicted signal'] + stocks['Negative Predicted signal']
    return stocks



In [84]:
#calculating volatility, however there is an error

# for idx, each_row in df.iterrows():
#     price=each_row['UnderlyingPrice']
#     strike=each_row['StrikePrice']
#     interest_rate=0
#     time_delta=each_row['time delta'].days
#     call_price=each_row['AskPrice']
#     implied_vol=mibian.BS([price, strike, interest_rate, time_delta], callPrice=call_price).impliedVolatility
#     df.loc[idx, 'implied_vol']=implied_vol
#     # break
